<a href="https://colab.research.google.com/github/asdfasdf001234/2024-1-MLPRJ/blob/main/googlenet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
import copy
import pandas as pd
from glob import glob

In [16]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/24-1 기계학습 팀프로젝트/img_model_test'

In [17]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [18]:
def create_dataframe(data_path, label_list):
    df = pd.DataFrame(columns=["path", "label", "class_id"])
    img_list = glob(os.path.join(data_path, '*.jpg'))

    for img in img_list:
        file_name = os.path.splitext(os.path.basename(img))[0]
        label_index = int(file_name[0]) - 1  # 파일 이름의 첫 번째 문자를 int로 변환하여 라벨 인덱스를 결정
        if 0 <= label_index < len(label_list):
            label = label_list[label_index]
            new_data = pd.DataFrame({"path": [img], "label": [label], "class_id": [label_index]})
            df = pd.concat([df, new_data], ignore_index=True)

    df["path"] = df["path"].astype(str)
    df["label"] = df["label"].astype(str)
    df["class_id"] = df["class_id"].astype(int)

    return df

# Example usage
data_path = "/path_to_your_data_directory"
train_path = os.path.join(data_path, 'Training')
valid_path = os.path.join(data_path, 'Validation')
test_path = os.path.join(data_path, 'Testing')
label_list = ['spring', 'summer', 'fall', 'winter']

train_df = create_dataframe(train_path, label_list)
val_df = create_dataframe(valid_path, label_list)
test_df = create_dataframe(test_path, label_list)



In [19]:
#데이터로더
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['test', 'train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [20]:
#장치 설정 - GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
#구글넷 모델을 불러오기
model = models.googlenet(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4개의 퍼스널 컬러 클래스
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
#loss 및 optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [23]:
def train_model(model, criterion, optimizer, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(best_model_wts)
    return model



In [24]:
# 모델 학습
model = train_model(model, criterion, optimizer, num_epochs=5)

# 모델 저장
torch.save(model.state_dict(), 'personal_color_googlenet.pth')

# 테스트 데이터로 모델 평가
model.eval()
test_loss = 0.0
test_corrects = 0

for inputs, labels in dataloaders['test']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    test_loss += loss.item() * inputs.size(0)
    test_corrects += torch.sum(preds == labels.data)

test_loss = test_loss / dataset_sizes['test']
test_acc = test_corrects.double() / dataset_sizes['test']

print(f'Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}')

Epoch 0/4
----------
train Loss: 0.6494 Acc: 0.8250
val Loss: 0.1122 Acc: 1.0000

Epoch 1/4
----------
train Loss: 0.0177 Acc: 1.0000
val Loss: 0.0064 Acc: 1.0000

Epoch 2/4
----------
train Loss: 0.0025 Acc: 1.0000
val Loss: 0.0021 Acc: 1.0000

Epoch 3/4
----------
train Loss: 0.0013 Acc: 1.0000
val Loss: 0.0014 Acc: 1.0000

Epoch 4/4
----------
train Loss: 0.0009 Acc: 1.0000
val Loss: 0.0011 Acc: 1.0000

Best val Acc: 1.000000
Test Loss: 0.1172 Acc: 1.0000
